In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install python-terrier==0.10.0 nltk scikit-learn lightgbm

In [ ]:
import pandas as pd
import json
import pyterrier as pt

In [ ]:
# Paths to JSONL files
jsonl_train_path = './text_data/tvr_train_release.jsonl'
jsonl_val_path = './text_data/tvr_val_release.jsonl'
subs_path = '.text_data/tvqa_preprocessed_subtitles.jsonl'

In [ ]:
# Paths to JSONL files on Colab
jsonl_train_path = '/content/drive/MyDrive/IR/text_data/tvr_train_release.jsonl'
jsonl_val_path = '/content/drive/MyDrive/IR/text_data/tvr_val_release.jsonl'
subs_path = '/content/drive/MyDrive/IR/text_data/tvqa_preprocessed_subtitles.jsonl'

In [ ]:
# Load subtitles into a dictionary for quick access
subtitles_dict = {}
with open(subs_path, 'r') as subs_file:
    for line in subs_file:
        sub_data = json.loads(line)
        subtitles_dict[sub_data['vid_name']] = sub_data['sub']

In [ ]:
# Function to find matching subtitles
def find_matching_subtitles(vid_name, ts_range, subtitles_dict):
    matching_subs = []
    if vid_name in subtitles_dict:
        for subtitle in subtitles_dict[vid_name]:
            if (ts_range[0] <= subtitle['start'] <= ts_range[1]) or (ts_range[0] <= subtitle['end'] <= ts_range[1]):
                matching_subs.append(subtitle['text'])
    return matching_subs

In [ ]:
def parse_jsonl(jsonl_path):
    # Initialize empty lists for your data
    queries_data = []
    documents_data = []
    query_rankings_data = []

    with open(jsonl_path, 'r') as file:
        for idx, line in enumerate(file):
            data = json.loads(line)

            # Extract data for the Query Set DataFrame
            queries_data.append({'qid': data['desc_id'], 'query': data['desc']})

            # Find matching subtitles
            matching_subs = find_matching_subtitles(data['vid_name'], data['ts'], subtitles_dict)

            # Extract data for the Documents Set DataFrame, including matching subtitles
            documents_data.append({'docno': idx, 'vid_name': data['vid_name'], 'ts': data['ts'],
                                'duration': data['duration'], 'type': data['type'], 'subtitles': matching_subs})

            # Extract data for the Query Rankings DataFrame
            query_rankings_data.append({'qid': data["desc_id"], 'query': data['desc'], 'docno': idx, 'rank': 1, 'score': 1.0})

    return queries_data, documents_data, query_rankings_data

In [ ]:
train_queries_data, train_documents_data, train_query_rankings_data = parse_jsonl(jsonl_train_path)

# Convert lists to DataFrames (training)
train_query_set_df = pd.DataFrame(train_queries_data).set_index('qid')
train_documents_set_df = pd.DataFrame(train_documents_data).set_index('docno')
train_query_rankings_df = pd.DataFrame(train_query_rankings_data)

val_queries_data, val_documents_data, val_query_rankings_data = parse_jsonl(jsonl_val_path)

# Convert lists to DataFrames (validation)
val_query_set_df = pd.DataFrame(val_queries_data).set_index('qid')
val_documents_set_df = pd.DataFrame(val_documents_data).set_index('docno')
val_query_rankings_df = pd.DataFrame(val_query_rankings_data)


In [ ]:
#print the head of the dataframes
print(train_query_set_df.head())
print(train_documents_set_df.head())
print(train_query_rankings_df.head())

print(val_query_set_df.head())
print(val_documents_set_df.head())
print(val_query_rankings_df.head())


In [ ]:
print(train_query_set_df)

### First Stage Retrieval [TODO: BOX]
The following part of the code will define three different first stage retrieval pipelines as an input for the trained model.

In [ ]:
if not pt.started():
    pt.init()

In [ ]:
# prepare the training data
if 'docno' not in train_documents_set_df.columns:
    train_documents_set_df.reset_index(inplace=True)

train_documents_set_df['text'] = train_documents_set_df['subtitles'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
train_documents_set_df['docno'] = train_documents_set_df['docno'].astype(str)

In [ ]:
# Create an index
indexer = pt.DFIndexer("/content/index_path", overwrite=True, meta=['docno'])
indexed = indexer.index(train_documents_set_df['text'], train_documents_set_df['docno'])

In [ ]:
# Initialize BatchRetrieve with the created index and specify BM25 as the weighting model
first_stage_bm25 = pt.BatchRetrieve(
    indexed,
    wmodel="BM25",
    num_results=100,
    metadata=["docno"]
)

Computing feature

In [ ]:
# features, can use any of the features in the list
pl2_retriever = pt.BatchRetrieve(indexed, wmodel="PL2", metadata=["docno"])
dph_retriever = pt.BatchRetrieve(indexed, wmodel="DPH", metadata=["docno"])

In [ ]:
# build a pipeline with the features
pipeline_with_features = ~first_stage_bm25 >> (
    pl2_retriever ** dph_retriever
)

In [ ]:
# output the results of dataset and query, help to overview the data

# train_documents_set_df = train_documents_set_df.dropna(subset=['text'])
# excel_filepath = 'documents.xlsx'

# train_documents_set_df.to_excel(excel_filepath, index=False, engine='openpyxl')

In [ ]:
# Prepare the queries for the pipeline, remove special characters and extra spaces
prepared_queries = train_query_set_df.reset_index()
prepared_queries['qid'] = prepared_queries['qid'].astype(str)
prepared_queries['query'] = prepared_queries['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_queries['query'] = prepared_queries['query'].str.replace(r'[^\w\s]', ' ', regex=True)

prepared_queries['query'] = prepared_queries['query'].str.replace(r'\s+', ' ', regex=True).str.strip()



In [ ]:
# divided in batch
from tqdm import tqdm

batch_size = 100
n_batches = (len(prepared_queries) + batch_size - 1) // batch_size

for batch_num in tqdm(range(n_batches), desc="Processing batches"):
    start_idx = batch_num * batch_size
    end_idx = start_idx + batch_size
    batch_queries = prepared_queries[start_idx:end_idx]

    # run pipeline
    batch_results = pipeline_with_features(batch_queries)

    # store file
    if batch_num == 0:
        # first batch
        batch_results.to_excel('results_with_features.xlsx', index=False, engine='openpyxl')
    else:
        # other batch
        with pd.ExcelWriter('results_with_features.xlsx', mode='a', engine='openpyxl', if_sheet_exists='overlay') as writer:
            batch_results.to_excel(writer, startrow=writer.sheets['Sheet1'].max_row, index=False, header=False)


In [ ]:
# directly run the pipeline
results_with_features = pipeline_with_features(prepared_queries)



### Learning [TODO: ROB]


#### Definition of learned rankers

In [ ]:
from sklearn.svm import SVR

learned_models = [
    SVR()
]

#### Rankers training

In [ ]:
trained_models = []
names = []

for fsr_pipeline in fsr_pipelines:
    for model in learned_models:
        names.append(f"{fsr_pipeline.name()}_{model.__class__.__name__}")
        pipe = ~fsr_pipeline >> pt.ltr.apply_learned_model(model)
        pipe.fit(
            train_query_rankings_df,
            train_query_set_df,
            train_documents_set_df
        )
        trained_models.append(pipe)

#### Rankers evaluation

In [ ]:
from pyterrier.measures import nDCG, RR, MAP

pt.Experiment(
    trained_models,
    val_query_set_df,
    val_query_rankings_df,
    names=names,
    eval_metrics=[nDCG @ 10, RR @ 10, MAP],
)